In [1]:
!apt-get install -y xvfb
!pip install Xvfbwrapper
!pip install pyvirtualdisplay
!pip install open3d

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.12).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from huggingface_hub import InferenceClient
import matplotlib.pyplot as plt

# Initialize the Inference Client with your Hugging Face token
client = InferenceClient("stabilityai/stable-diffusion-3.5-large-turbo", token="hf_YourToken")

# Generate the image based on the text prompt
image = client.text_to_image("horse")

# Display the generated image
plt.imshow(image)
plt.savefig('texttoimage.png')



In [3]:
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation
from PIL import Image
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt



# Initialize model and processor
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

# Resize image for the model input
new_height = 480 if image.height > 480 else image.height
new_height -= new_height % 32
new_width = int(new_height * image.width / image.height)
diff = new_width % 32
new_width = new_width - diff if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

# Prepare the input for depth estimation
inputs = feature_extractor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

# Process the depth map
pad = 16
output = predicted_depth.squeeze().cpu().numpy() * 1000
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))

# Display depth map
fix, ax = plt.subplots(1, 2)
ax[0].imshow(image)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap='plasma')
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.savefig('depth_map.png')
plt.show()


In [4]:
import open3d as o3d
# Convert depth map to Open3D format
depth_image = (output * 255 / np.max(output)).astype('uint8')
image = np.array(image)
depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)

# Get the image as a numpy array (assuming image_o3d is an open3d Image object)
image_np = np.asarray(image_o3d)

# Get width and height from the numpy array's shape
height, width = image_np.shape[:2]  # height and width of the image

# Now, set the camera intrinsic parameters
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width / 2, height / 2)


# Generate point cloud
pcd_raw = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)
o3d.visualization.draw_geometries([pcd_raw])



# Process and visualize point cloud
cl, ind = pcd_raw.remove_statistical_outlier(nb_neighbors=20, std_ratio=6.0)
pcd = pcd_raw.select_by_index(ind)
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()

# Visualize point cloud with normals
o3d.visualization.draw_geometries([pcd])

# Create mesh from point cloud
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, n_threads=1, width=0, scale=1.1, linear_fit=False)[0]
rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))

# Visualize mesh
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

# Paint mesh with uniform color and save
mesh_uniform = mesh.paint_uniform_color([0.9, 0.8, 0.9])
mesh_uniform.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_uniform], mesh_show_back_face=True)
o3d.io.write_triangle_mesh("mesh.obj", mesh)


# Save point cloud
o3d.io.write_point_cloud("point_cloud.ply", pcd_raw)



[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] Write OBJ can not include triangle normals.


True